# Assignment 1

Explain

## Olympics
---
Todo

In [3]:
1+1

2

## Business Understanding
---
Todo

In [4]:
1+1

2

## Data Meaning Type
---
Todo

In [5]:
1+1

2

## Data Quality
---
Todo

In [6]:
1+1

2

## Simple Statistics
---
Todo

In [7]:
1+1

2

## Visualize Attributes
---
Todo

In [8]:
1+1

2

## Explore Joint Attributes
---
Todo

In [9]:
1+1

2

## Explore Attributes and Class
---
Todo

In [10]:
1+1

2

## New Features
---
Todo

In [11]:
1+1

2

## Exceptional Work
---
Todo

In [12]:
1+1

2